In [1]:
from pathlib import Path

import polars as pl
from aoc.polars_utils import pl_group_all

data = Path("../Data/day1.txt").read_text()

EXAMPLE = """
3   4
4   3
2   5
1   3
3   9
3   3
"""


def parse(input: str):
    def map_line(line: str):
        splitten_line = line.split("   ")

        try:
            return {"left": int(splitten_line[0]), "right": int(splitten_line[1])}
        except ValueError:
            return {"left": None, "right": None}

    return (
        pl.LazyFrame(
            map(map_line, input.splitlines()), {"left": pl.Int64, "right": pl.Int64}
        )
        .filter(pl.col("left") is not None)
        .with_columns(
            pl.col("left").sort().alias("left_sorted"),
            pl.col("right").sort().alias("right_sorted"),
        )
        .drop("left", "right")
        .rename({"left_sorted": "left", "right_sorted": "right"})
    )


example_data_frame = parse(EXAMPLE)
data_frame = parse(data)

In [2]:
def part1(input: pl.LazyFrame):
    return pl_group_all(
        input.with_columns(
            (pl.col("left") - pl.col("right")).abs().alias("difference")
        ),
        lambda grouped_data_frame: grouped_data_frame.agg(pl.col("difference").sum()),
    ).collect()


assert part1(example_data_frame).get_column("difference")[0] == 11

part1(data_frame)

difference
i64
1882714


In [3]:
def part2(input: pl.LazyFrame):
    return pl_group_all(
        input.select("right")
        .group_by("right")
        .agg(pl.col("right").count().alias("count"))
        .with_columns(pl.col("right").alias("left"))
        .join(input.select("left"), on="left")
        .with_columns((pl.col("count") * pl.col("left")).alias("score")),
        lambda grouped_data_frame: grouped_data_frame.agg(pl.col("score").sum()),
    ).collect()


assert part2(example_data_frame).get_column("score")[0] == 31

part2(data_frame)

score
i64
19437052
